<a href="https://colab.research.google.com/github/M-Tech-AI-24MAI012/Summer-Internship-PUF-doc/blob/main/New_of_PUF_GBNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F


from sklearn.base import BaseEstimator

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
import os
from google.colab import drive

if not os.path.exists("/content/db1"):
    drive.mount('/content/drive')
    !unzip "/content/drive/MyDrive/Summer Internship/ML Attack.zip" -d "/content/db1"
else:
    print("Folder '/content/db1' already exists. Skipping unzip.")


Folder '/content/db1' already exists. Skipping unzip.


# Load the Excel files into pandas DataFrames

In [ ]:
# Load the Excel files into pandas DataFrames

response_delay_path = "/content/db1/ML Attack/Responce_Delay_2us.xlsx"
challenge_path = "/content/db1/ML Attack/Challenge.xlsx"
response_delay_sheet_names = pd.ExcelFile(response_delay_path).sheet_names
response_delay_df = pd.read_excel(response_delay_path, sheet_name=0,header=None)

challenge_df = pd.read_excel(challenge_path,header=None)


In [ ]:
# Display the first few rows of each DataFrame to verify
print("Selected Sheet Name:", response_delay_sheet_names)

print("Response Delay DataFrame:")
display(response_delay_df.head())

print("\nChallenge DataFrame:")
display(challenge_df.head())

Selected Sheet Name: ['FD_MODE_1', 'VD_MODE_1', 'FD_MODE_2', 'VD_MODE_2', 'FD_MODE_3', 'VD_MODE_3', 'FD_MODE_4', 'VD_MODE_4', 'FD_MODE_5', 'VD_MODE_5', 'FD_MODE_6', 'VD_MODE_6', 'FD_MODE_7', 'VD_MODE_7', 'FD_MODE_8', 'VD_MODE_8']
Response Delay DataFrame:


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,1,1,1,1,0,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
1,1,0,1,1,1,0,0,0,0,1,...,0,1,1,0,0,0,1,0,0,1
2,1,1,0,0,1,1,0,0,0,0,...,1,1,0,1,1,0,1,1,1,1
3,1,1,1,0,1,1,0,0,1,1,...,1,1,0,0,0,1,1,0,1,1
4,0,1,0,0,0,1,0,1,1,1,...,1,0,1,1,0,0,0,1,0,0



Challenge DataFrame:


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,1,0,0,0,0,0,1,1,1,...,1,0,1,1,0,0,0,1,1,1
1,0,1,1,0,0,1,0,1,1,1,...,0,1,1,0,1,0,0,1,0,0
2,0,0,1,0,1,0,0,1,0,0,...,0,0,1,1,0,1,1,0,1,1
3,0,0,0,1,1,0,0,0,1,0,...,1,1,0,0,1,0,0,0,1,1
4,1,1,1,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,1,0


In [ ]:
# Convert to NumPy
X = challenge_df.values.astype(np.float32)     # shape: (N, 32)
Y = response_delay_df.values.astype(np.float32)  # shape: (N, 32)

In [ ]:
X

array([[0., 1., 0., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 1., 1.],
       ...,
       [0., 1., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [ ]:
Y

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       ...,
       [0., 1., 0., ..., 1., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 1., 0., ..., 1., 1., 1.]], dtype=float32)

In [ ]:
#X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:

import re


def get_next_model_version(base_path="/content/drive/MyDrive/SummerIntern/Puf Models", prefix="puf_gbnn"):
    os.makedirs(base_path, exist_ok=True)
    existing = [f for f in os.listdir(base_path) if f.startswith(prefix)]
    versions = [int(re.findall(r'V(\d+)', name)[0]) for name in existing if re.search(r'V\d+', name)]
    next_version = max(versions) + 1 if versions else 1
    model_folder = f"{prefix}_V{next_version}"
    return os.path.join(base_path, model_folder)

def save_gbnn_model(model, path, history=None):
    os.makedirs(path, exist_ok=True)
    model_path = os.path.join(path, "model.pt")
    torch.save(model.state_dict(), model_path)
    print(f"✅ GBNN model saved at: {model_path}")

    if history is not None:
        history_df = pd.DataFrame(history, columns=[
            "Stage", "Epoch", "Train Accuracy", "Val Accuracy"
        ])
        excel_path = os.path.join(path, "training_history.xlsx")
        history_df.to_excel(excel_path, index=False)
        print(f" Training history saved at: {excel_path}")

def load_gbnn_model(model_class, model_args, path):
    model_path = os.path.join(path, "model.pt")
    model = model_class(**model_args)
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    print(f"✅ GBNN model loaded from: {model_path}")
    return model


In [ ]:


# MLP for each stage

class MultiOutputNN(nn.Module):
    def __init__(self):
        super(MultiOutputNN, self).__init__()
        self.model = nn.Sequential(

            nn.Linear(32, 5000),
            nn.BatchNorm1d(5000),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),

            nn.Linear(5000, 2048),
            nn.BatchNorm1d(2048),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),


            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),


            nn.Linear(1024, 512),
            nn.LeakyReLU(0.1),

            nn.Linear(512, 256),
            nn.LeakyReLU(0.1),

            nn.Linear(256, 64),
            nn.LeakyReLU(0.1),

            nn.Linear(64, 32)
        )

    def forward(self, x):
        return self.model(x)

def bitwise_accuracy(preds, labels):
    return (preds == labels).float().mean().item()

def exact_match(preds, labels):
    return ((preds == labels).all(dim=1)).float().mean().item()



class GBNN(nn.Module):
    def __init__(self, n_stages=30):
        super(GBNN, self).__init__()
        self.n_stages = n_stages
        self.models = nn.ModuleList([MultiOutputNN() for _ in range(n_stages)])

    def forward(self, x):
        out = torch.zeros(x.size(0), 32).to(x.device)
        for model in self.models:
            out += model(x)
        return out

    def stagewise_forward(self, x, up_to_stage):
        out = torch.zeros(x.size(0), 32).to(x.device)
        for model in self.models[:up_to_stage]:
            out += model(x)
        return out

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
import pandas as pd

def line_search_rho(logits_prev, h_out, y_true, n_steps=20):
    min_loss = float('inf')
    best_rho = 1.0
    criterion = nn.BCEWithLogitsLoss()

    for rho in torch.linspace(0.01, 2.0, steps=n_steps):
        logits = logits_prev + rho * h_out
        loss = criterion(logits, y_true)
        if loss.item() < min_loss:
            min_loss = loss.item()
            best_rho = rho.item()
    return best_rho

def split_dataset(dataset):
    total_size = len(dataset)
    train_size = int(0.7 * total_size)
    test_size = int(0.2 * total_size)
    val_size = total_size - train_size - test_size
    return random_split(dataset, [train_size, test_size, val_size])

def train_gbnn(model, X_train, y_train, X_val, y_val, X_test, y_test, n_epochs=100, lr=0.01, save_path=None):
    model = model.to(device)
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_val, y_val = X_val.to(device), y_val.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)

    history = []
    rhos = []

    # --- Initialize base prediction (F0) ---
    y_mean = y_train.mean(dim=0).clamp(1e-5, 1 - 1e-5)
    F0 = torch.log(y_mean / (1 - y_mean))  # inverse sigmoid (logit)
    base_train = F0.unsqueeze(0).expand(X_train.size(0), -1).clone().to(device)
    base_val   = F0.unsqueeze(0).expand(X_val.size(0), -1).clone().to(device)
    base_test  = F0.unsqueeze(0).expand(X_test.size(0), -1).clone().to(device)

    def compute_pseudo_residuals(logits, y_true):
        probs = torch.sigmoid(logits)
        return y_true - probs  # Negative gradient of BCE loss

    def line_search_rho(f_prev, h, y_true):
        def loss_fn(rho):
            logits = f_prev + rho * h
            return nn.BCEWithLogitsLoss()(logits, y_true)
        best_rho = 1.0
        best_loss = loss_fn(torch.tensor(best_rho).to(device))
        for rho in torch.linspace(0.1, 2.0, steps=20).to(device):
            loss = loss_fn(rho)
            if loss < best_loss:
                best_loss = loss
                best_rho = rho.item()
        return best_rho

    model_outputs = base_train.clone()

    # --- Training loop ---
    for stage_idx, sub_model in enumerate(model.models):
        print(f"\n🎯 Stage {stage_idx + 1}/{model.n_stages}")
        optimizer = optim.Adam(sub_model.parameters(), lr=lr, weight_decay=1e-5)

        for epoch in range(n_epochs):
            model.train()
            with torch.no_grad():
                f_prev = model_outputs.clone()
                residuals = compute_pseudo_residuals(f_prev, y_train)

            pred_residuals = sub_model(X_train)
            loss = nn.MSELoss()(pred_residuals, residuals)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                h = sub_model(X_train)
                f_prev = model_outputs.clone()
                rho = line_search_rho(f_prev, h, y_train)
                model_outputs = f_prev + rho * h

            # --- Evaluation ---
            model.eval()
            with torch.no_grad():
                train_preds = (torch.sigmoid(model_outputs) > 0.5).float()
                train_acc = bitwise_accuracy(train_preds, y_train)

                val_logits = base_val.clone()
                for i in range(stage_idx):
                    val_logits += rhos[i] * model.models[i](X_val)
                val_logits += rho * model.models[stage_idx](X_val)

                val_preds = (torch.sigmoid(val_logits) > 0.5).float()
                val_acc = bitwise_accuracy(val_preds, y_val)

            history.append([
                stage_idx + 1,
                epoch + 1,
                float(train_acc),
                float(val_acc),
                float(loss.item())
            ])

            if (epoch % 100 == 0) or (epoch == n_epochs - 1):
                print(f"Epoch {epoch+1}/{n_epochs} | Loss: {loss.item():.4f} | Train Acc: {train_acc*100:.2f}% | Val Acc: {val_acc*100:.2f}%")

            # ⛔ Early stop inner epoch loop
            if train_acc == 1.0:
                print(f"🎉 Stopping early at Epoch {epoch+1} — 100% Training Accuracy Achieved!")
                break

        rhos.append(rho)

        # ⛔ Early stop outer stage loop
        if train_acc == 1.0:
            print(f"🚀 Stopping GBNN training after stage {stage_idx+1} — 100% Training Accuracy achieved!")
            break

    # --- Save model and training history ---
    if save_path is not None:
        os.makedirs(save_path, exist_ok=True)
        torch.save(model.state_dict(), os.path.join(save_path, "model.pt"))
        print(f"✅ Model saved at: {os.path.join(save_path, 'model.pt')}")
        df = pd.DataFrame(history, columns=["Stage", "Epoch", "Train Accuracy", "Val Accuracy", "Train Error"])
        df.to_excel(os.path.join(save_path, "training_history.xlsx"), index=False)
        print(f"📄 Training history saved at: {os.path.join(save_path, 'training_history.xlsx')}")

    # --- Final Test Performance ---
    model.eval()
    with torch.no_grad():
        test_logits = base_test.clone()
        for i in range(len(rhos)):  # Only stages that were actually trained
            test_logits += rhos[i] * model.models[i](X_test)
        test_preds = (torch.sigmoid(test_logits) > 0.5).float()
        test_acc = bitwise_accuracy(test_preds, y_test)
        test_exact = exact_match(test_preds, y_test)

    print(f"\n✅ Test Acc: {test_acc*100:.2f}% | Test Exact Match: {test_exact*100:.2f}%")

    return model, history


In [ ]:


save_path = get_next_model_version(
    base_path="Puf Models",
    prefix="puf_gbnn"
)
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, Y_tensor)
train_set, test_set, val_set = split_dataset(dataset)

train_data = next(iter(DataLoader(train_set, batch_size=64)))
test_data = next(iter(DataLoader(test_set, batch_size=64)))
val_data = next(iter(DataLoader(val_set, batch_size=64)))

# Unpack data
X_train, y_train = train_data
X_val, y_val = val_data
X_test, y_test = test_data
# Train and save
model = GBNN(n_stages=100)
trained_model, history = train_gbnn(model, X_train, y_train, X_val, y_val, X_test, y_test, n_epochs=100, lr=0.01, save_path=save_path)


🎯 Stage 1/100
Epoch 1/100 | Loss: 0.2484 | Train Acc: 58.40% | Val Acc: 53.08%
Epoch 100/100 | Loss: 0.1847 | Train Acc: 73.97% | Val Acc: 53.27%

🎯 Stage 2/100
Epoch 1/100 | Loss: 0.1848 | Train Acc: 74.12% | Val Acc: 52.98%
Epoch 100/100 | Loss: 0.1480 | Train Acc: 79.05% | Val Acc: 54.64%

🎯 Stage 3/100
Epoch 1/100 | Loss: 0.1479 | Train Acc: 79.93% | Val Acc: 52.78%
Epoch 100/100 | Loss: 0.2648 | Train Acc: 86.28% | Val Acc: 53.76%

🎯 Stage 4/100
Epoch 1/100 | Loss: 0.1067 | Train Acc: 85.99% | Val Acc: 54.00%
Epoch 100/100 | Loss: 0.0840 | Train Acc: 89.31% | Val Acc: 53.12%

🎯 Stage 5/100
Epoch 1/100 | Loss: 0.0845 | Train Acc: 89.60% | Val Acc: 52.78%
Epoch 100/100 | Loss: 0.0691 | Train Acc: 91.26% | Val Acc: 53.42%

🎯 Stage 6/100
Epoch 1/100 | Loss: 0.0732 | Train Acc: 91.50% | Val Acc: 53.32%
Epoch 100/100 | Loss: 0.0580 | Train Acc: 92.82% | Val Acc: 53.37%

🎯 Stage 7/100
Epoch 1/100 | Loss: 0.0594 | Train Acc: 93.07% | Val Acc: 53.42%
Epoch 100/100 | Loss: 0.0389 | Train A

In [ ]:
'''

X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, Y_tensor)
train_set, test_set, val_set = split_dataset(dataset)

train_data = next(iter(DataLoader(train_set, batch_size=len(train_set))))
test_data = next(iter(DataLoader(test_set, batch_size=len(test_set))))
val_data = next(iter(DataLoader(val_set, batch_size=len(val_set))))

# Unpack data
X_train, y_train = train_data
X_val, y_val = val_data
X_test, y_test = test_data

# --- Train ---
model = GBNN(n_stages=35)
train_gbnn(model, X_train, y_train, X_val, y_val, X_test, y_test, n_epochs=200, lr=0.01)

'''


In [ ]:

# Predict 32-bit output from 32-bit input
def predict_single_32bit(model: nn.Module, input_bits: list, threshold: float = 0.5) -> list:
    assert len(input_bits) == 32, "Input must be a 32-bit binary list."

    model.eval()
    input_array = np.array(input_bits, dtype=np.float32)
    normalized_input = input_array * 0.98 + 0.01

    with torch.no_grad():
        input_tensor = torch.tensor(normalized_input).unsqueeze(0)  # Shape: [1, 32]
        logits = model(input_tensor)
        probs = torch.sigmoid(logits)
        prediction = (probs > threshold).int().squeeze().tolist()

    return prediction

def predict_batch_32bit(model: nn.Module, input_array: np.ndarray, threshold: float = 0.5) -> list:
    assert input_array.ndim == 2 and input_array.shape[1] == 32, "Input must be of shape [N, 32]"

    model.eval()
    normalized_input = input_array * 0.98 + 0.01

    with torch.no_grad():
        input_tensor = torch.tensor(normalized_input, dtype=torch.float32)
        logits = model(input_tensor)
        probs = torch.sigmoid(logits)
        predictions = (probs > threshold).int().tolist()  # List of [0,1] lists

    return predictions


def evaluate_predictions(y_true: np.ndarray, y_pred: np.ndarray):

    assert y_true.shape == y_pred.shape, "Shape mismatch between true and predicted labels"

    metrics = {}

    # Bitwise metrics (macro = average over each bit)
    metrics["bitwise_accuracy"] = accuracy_score(y_true.flatten(), y_pred.flatten())
    metrics["macro_precision"] = precision_score(y_true, y_pred, average='macro', zero_division=0)
    metrics["macro_recall"] = recall_score(y_true, y_pred, average='macro', zero_division=0)
    metrics["macro_f1"] = f1_score(y_true, y_pred, average='macro', zero_division=0)

    # Hamming loss (fraction of incorrect bits)
    metrics["hamming_loss"] = hamming_loss(y_true, y_pred)

    # Exact match (all 32 bits correct)
    exact_matches = np.all(y_true == y_pred, axis=1)
    metrics["exact_match_ratio"] = np.mean(exact_matches)

    return metrics


In [ ]:

# Predict a 32-bit binary input
input_bits = [0, 1, 0, 1, 0, 1, 1, 0,
              1, 0, 0, 1, 1, 0, 0, 1,
              1, 1, 0, 0, 1, 0, 1, 0,
              1, 1, 0, 0, 1, 0, 0, 1]

predicted_output = predict_single_32bit(model, input_bits)
print("\nPredicted Output Bits:", predicted_output)

In [ ]:
X = challenge_df.values.astype(np.float32)  # shape [N, 32]

# Predict all rows
predictions = predict_batch_32bit(model, X)


predictions_np = np.array(predictions)

print("Predictions shape:", predictions_np.shape)
print("First prediction:", predictions[0])

In [ ]:

Y_true = Y.astype(np.int32)
X_input = X.astype(np.float32)
Y_pred = np.array(predict_batch_32bit(model, X_input))

# Evaluate
metrics = evaluate_predictions(Y_true, Y_pred)


for name, value in metrics.items():
    print(f"{name}: {value:.4f}")


In [ ]:

def top_k_bit_errors(y_true, y_pred, k=5):


    bit_errors_per_sample = np.sum(y_true != y_pred, axis=1)


    top_k_indices = np.argsort(bit_errors_per_sample)[-k:][::-1]

    top_k_errors = bit_errors_per_sample[top_k_indices]

    return top_k_indices, top_k_errors


top_indices, top_errors = top_k_bit_errors(Y_true, Y_pred, k=5)

print("Top 5 samples with highest bit errors:")
for rank, (idx, errors) in enumerate(zip(top_indices, top_errors), 1):
    print(f"Rank {rank}: Sample index {idx}, Bit errors = {errors}")
    print(f"True bits: {Y_true[idx]}")
    print(f"Pred bits: {Y_pred[idx]}")
    print("-" * 30)